# Storage Utilization Heatmap

In [ ]:
import nsys_display
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as pyo

from IPython.display import HTML, Markdown, display
from plotly.subplots import make_subplots

display(HTML('<style>.container { width:95% !important; }</style>'))
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", None)
pyo.init_notebook_mode()

This line graph displays the summary of all operations on NFS, Lustre, Local and NVMe-oF file systems
for the profiled volumes:
* x axis represents the rank duration, scaling from 0 to the maximum duration across all ranks.
* y axis represents the mean Bytes read and written across all ranks.

In [ ]:
# Load the DataFrame from the parquet file
df = pd.read_parquet("analysis.parquet")

# Create unique name.
df["Name"] = (
    df["Rank"].astype(str)
    + "/"
    + df["Hostname"].astype(str)
    + "/"
    + df["Volume"].astype(str)
    + "/"
    + df["Name"].astype(str)
)

# Convert ns to seconds
df["Duration"] = df["Duration"] * 1e-9

all_metrics = ["Read", "Write"]

nsys_display.display_summary_graph(
    df,
    all_metrics,
    xaxis_title="Duration (s)",
    yaxis_title="Value",
    title="Usage Summary (bins=REPLACE_BIN)",
)

These heatmaps display NFS, Lustre, Local and NVMe-oF filesystem operations which are collected using NVTX counters
via the storage-metrics plugin and the --storage-metrics feature
* x axis represents the rank duration, scaling from 0 to the maximum duration across all ranks.
* y axis represents the set of Rank/Hostname/Volume/Device name for which metrics were collected.

The heatmaps present:
* Bytes read
* Bytes written

In [ ]:
# Create heatmaps for each metric.
for operation in all_metrics:
    if operation not in df.columns:
        print(f"Data for '{operation}' is not in the DataFrame.")
        continue

    fig = make_subplots(
        rows=1, cols=1, subplot_titles=[operation], vertical_spacing=0.1
    )

    # Add the heatmap trace
    fig.add_trace(
        go.Heatmap(
            x=df["Duration"],
            y=df["Name"],
            z=df[operation],
            colorscale="Viridis",
            zmax=df[operation].max(),
            zmin=df[operation].min(),
            colorbar=dict(title="Bytes"),
            hovertemplate=(
                '<br>'.join([
                    'Duration (s): %{{x}}',
                    'Source: %{{y}}',
                    'Bytes: %{{z}}'])+'<extra></extra>').format(type)),
        1,    1)

    # Layout update
    fig.update_layout(
        height=500,
        title=f"Heatmap of {operation}",
        xaxis_title="Duration (s)",
        yaxis_title="Name",
        xaxis=dict(
            title="Duration (s)", titlefont=dict(size=14), tickfont=dict(size=12)
        ),
        yaxis=dict(
            title="Rank/Hostname/Volume/Device",
            titlefont=dict(size=14),
            tickfont=dict(size=12),
            categoryorder="category descending",
        )
    )

    fig.show()

# Files

The table associates each rank number with the original filename. Ranks are assigned assuming that
the file names include the rank with sufficient zero padding for proper sorting. Otherwise, the
actual rank may differ from the assigned ID.

In [ ]:
files_df = pd.read_parquet("files.parquet")
display(files_df)